As you about to start the road of terror by debuging this code here is the word of wisdom <b>"Go and read the README first!"</b>
<br>
There are explanations and examples you need to understand this deep enough to debug.

In [13]:
# import all libs
import numpy as np
import pandas as pd
import datetime as dt

## Getting data

In [14]:
# connect to DB
from helpers.util import connect
con, cur = connect('vertica')

By this moment we added only few formulas into the base, so for this example I took UE_THROUGHPUT_DL/UL

In [19]:
formulas = pd.read_sql('''select KPI_SHORT, VENDOR, FORMULA_NUM_DENOM, DATE_START 
from VERTICA_STAT.FORMULA where KPI_SHORT in ('UE_THROUGHPUT_DL', 'UE_THROUGHPUT_UL')
''', con, parse_dates=['DATE_START']) # 1st lvl formulas

num_den = pd.read_sql('''select upper(NUM_DEN) NUM_DEN, VENDOR, FORMULA, DATE_START, AG, STANDART 
from VERTICA_STAT.NUM_DENOM
where NUM_DEN in ('NUE_THROUGHPUT_DL_KBIT', 'NUE_THROUGHPUT_UL_KBIT', 'dUE_Throughput_DL', 'dUE_Throughput_UL')
and vendor = 'N' ''', con, parse_dates=['DATE_START'])  # 2nd lvl formulas

Initial data looks as follows

In [20]:
formulas

,KPI_SHORT,VENDOR,FORMULA_NUM_DENOM,DATE_START
0,UE_THROUGHPUT_DL,N,[NUE_THROUGHPUT_DL_KBIT / DUE_THROUGHPUT_DL],2018-01-01
1,UE_THROUGHPUT_UL,N,[NUE_THROUGHPUT_UL_KBIT / DUE_THROUGHPUT_UL],2018-01-01


In [21]:
num_den

,NUM_DEN,VENDOR,FORMULA,DATE_START,AG,STANDART
0,NUE_THROUGHPUT_UL_KBIT,N,(8*((M8012C91+M8012C93+M8012C95+M8012C97+M8012...,2018-01-01,SUM,4G
1,DUE_THROUGHPUT_UL,N,((M8012C156+M8012C157+M8012C158+M8012C159+M801...,2018-01-01,SUM,4G
2,NUE_THROUGHPUT_DL_KBIT,N,(8*((M8012C117+M8012C119+M8012C121+M8012C123+M...,2018-01-01,SUM,4G
3,DUE_THROUGHPUT_DL,N,((M8012C118+M8012C120+M8012C122+M8012C124+M801...,2018-01-01,SUM,4G
4,DUE_THROUGHPUT_DL,N,(M8012C165+M8012C166+M8012C167+M8012C168+M8012...,2021-03-10,SUM,4G


## The magic

The whole bunch of utilities I wrote for making this more automatised and places into lib, to make this file less messy. 
<br>
These are explained inside the library.

In [6]:
from helpers.vertica import create_cases, add_aggr_nvl, check_tree_for_instance, check_tree_for_replace_dividers

We start by going though all of the second level formulas and adding SUM(NVL(...,0)) to every counter inside
<br>
Then we create cases for evolution of every formula (case when time between a and b then c when ...), so we have all the changes of the formula thoughout its life. 
I save this cases into dict (not to well formated string) because later we are going to need to check the dates of existence of every separate version of the formula and it's rather difficult with string (before you ask, regex is not an option).

In [22]:
num_den['FIXED_FORMULAS'] = [add_aggr_nvl(i[1]['FORMULA'], i[1]['AG']) for i in num_den.iterrows()]
col1 = 'NUM_DEN' # source of counters in DATA
col2 = 'FIXED_FORMULAS' # source of formulas in DATA
counters = create_cases(col1, col2, num_den)
num_den['CASE'] = [counters[i] for i in num_den['NUM_DEN']]

<p>For this example the formulas looks like follows:</p>
<ul>
<li>   DUE_THROUGHPUT_UL, NUE_THROUGHPUT_UL_KBIT and NUE_THROUGHPUT_DL_KBIT where not changing -> exists in only one version.</li>
<li>   DUE_THROUGHPUT_DL changed couple of days ago.</li>
</ul>

In [8]:
counters

{'DUE_THROUGHPUT_UL': '((((((((((SUM(nvl(M8012C156,0))+SUM(nvl(M8012C157,0)))+SUM(nvl(M8012C158,0)))+SUM(nvl(M8012C159,0)))+SUM(nvl(M8012C160,0)))+SUM(nvl(M8012C161,0)))+SUM(nvl(M8012C162,0)))+SUM(nvl(M8012C163,0)))+SUM(nvl(M8012C164,0)))/1000)',
 'NUE_THROUGHPUT_UL_KBIT': '(8*((((((((((SUM(nvl(M8012C91,0))+SUM(nvl(M8012C93,0)))+SUM(nvl(M8012C95,0)))+SUM(nvl(M8012C97,0)))+SUM(nvl(M8012C99,0)))+SUM(nvl(M8012C101,0)))+SUM(nvl(M8012C103,0)))+SUM(nvl(M8012C105,0)))+SUM(nvl(M8012C107,0)))/(8*1000)))',
 'NUE_THROUGHPUT_DL_KBIT': '(8*((((((((((SUM(nvl(M8012C117,0))+SUM(nvl(M8012C119,0)))+SUM(nvl(M8012C121,0)))+SUM(nvl(M8012C123,0)))+SUM(nvl(M8012C125,0)))+SUM(nvl(M8012C127,0)))+SUM(nvl(M8012C129,0)))+SUM(nvl(M8012C131,0)))+SUM(nvl(M8012C133,0)))/(8*1000)))',
 'DUE_THROUGHPUT_DL': {'case': {'when': [[{'between': {'counter': 'datetime',
       'date_start': datetime.date(2021, 3, 10),
       'date_end': datetime.date(2021, 3, 11),
       'by_date': True}},
     '((((((((((SUM(nvl(M8012C165,0))+

So by this moment we created rules for the second level formulas. Looks easy, huh?

In [9]:
def find_end_date(data, dates, kpi):
    temp = formulas.loc[data['KPI_SHORT']==kpi]
    temp = temp['DATE_START']
    temp = sorted(temp)
    ind = temp.index(dates)
    if ind < len(temp)-1:
        return(temp[ind+1].date()-dt.timedelta(days=1) )
    else:
        return(dt.datetime.today().date()) 

Here we are adding the end dates for every formula version.

In [23]:
formulas['END_DATE'] = [find_end_date(formulas, i[1]['DATE_START'], i[1]['KPI_SHORT']) for i in formulas.iterrows()]

And here we go to the first level formulas and change first level counters into our cases structure.

In [24]:
formulas['FIXED_FORMULAS'] = [check_tree_for_replace_dividers(i[1]['FORMULA_NUM_DENOM'],
                                                                i[1]['DATE_START'], i[1]['END_DATE'],
                                                                counters) for i in formulas.iterrows()]

In [25]:
formulas.head()

,KPI_SHORT,VENDOR,FORMULA_NUM_DENOM,DATE_START,END_DATE,FIXED_FORMULAS
0,UE_THROUGHPUT_DL,N,[NUE_THROUGHPUT_DL_KBIT / DUE_THROUGHPUT_DL],2018-01-01,2021-03-11,ONLINESTAT.F_DEV((8*((((((((((SUM(nvl(M8012C11...
1,UE_THROUGHPUT_UL,N,[NUE_THROUGHPUT_UL_KBIT / DUE_THROUGHPUT_UL],2018-01-01,2021-03-11,ONLINESTAT.F_DEV((8*((((((((((SUM(nvl(M8012C91...


In [26]:
formulas.loc[0, 'KPI_SHORT'], formulas.loc[0, 'FIXED_FORMULAS']

('UE_THROUGHPUT_DL',
 "ONLINESTAT.F_DEV((8*((((((((((SUM(nvl(M8012C117,0))+SUM(nvl(M8012C119,0)))+SUM(nvl(M8012C121,0)))+SUM(nvl(M8012C123,0)))+SUM(nvl(M8012C125,0)))+SUM(nvl(M8012C127,0)))+SUM(nvl(M8012C129,0)))+SUM(nvl(M8012C131,0)))+SUM(nvl(M8012C133,0)))/(8*1000))), case when datetime between to_date('2021-03-10', 'YYYY-MM-DD') and to_date('2021-03-11', 'YYYY-MM-DD') then ((((((((((SUM(nvl(M8012C165,0))+SUM(nvl(M8012C166,0)))+SUM(nvl(M8012C167,0)))+SUM(nvl(M8012C168,0)))+SUM(nvl(M8012C169,0)))+SUM(nvl(M8012C170,0)))+SUM(nvl(M8012C171,0)))+SUM(nvl(M8012C172,0)))+SUM(nvl(M8012C173,0)))/1000) when datetime between to_date('2018-01-01', 'YYYY-MM-DD') and to_date('2021-03-09', 'YYYY-MM-DD') then ((((((((((SUM(nvl(M8012C118,0))+SUM(nvl(M8012C120,0)))+SUM(nvl(M8012C122,0)))+SUM(nvl(M8012C124,0)))+SUM(nvl(M8012C126,0)))+SUM(nvl(M8012C128,0)))+SUM(nvl(M8012C130,0)))+SUM(nvl(M8012C132,0)))+SUM(nvl(M8012C134,0)))/1000)  else null end)")

All in all we got this. Doesn't it look pretty? It definitely does not. But it's the only adequate way of doing this.
Soon this beautiful creature will go on production and every day generate each and every formula for everyone.
<br>
Later, if needed, we can call create_cases again and create one big request for entire table.